<a href="https://colab.research.google.com/github/Aribabo/Matchuri/blob/main/%EB%AA%A8%EB%8D%B8_%EA%B2%B0%EA%B3%BC_%ED%99%95%EC%9D%B8%EC%9A%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 파일정보
* 유사도 측정을 적용한 모델 완전체 생성이 목표
* GPT2와 roBERTa를 사용하여 작성
* 모델 정보 : skt-kogpt2-base-v2-02070909-cuda.pth
* 데이터 셋 정보 : 손수 정제한 문제와 답, 키워드 사용


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 여기에서 데이터 통합본 path와 불러올 모델 path를 설정해주세요

In [ ]:
excel_path = '/content/drive/MyDrive/KDT-챗봇 프로젝트/모델실험실/데이터통합본-서버용.xlsx'
model_path = '/content/drive/MyDrive/KDT-챗봇 프로젝트/모델실험실/정흔/최종데이터모델/skt-kogpt2-base-v2-2024-02-13_3-cuda.pth'

# 데이터 로드

In [ ]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, AutoTokenizer, AutoModel
import re
import torch.nn.functional as F

데이터 로드


In [ ]:
excel_file = pd.ExcelFile(excel_path)
sheet_names = excel_file.sheet_names

df = pd.read_excel(excel_file, '문제답')
df

,새문제번호,원래번호,문제,정답,유사도문제,유사도답,유사도키워드
0,1,1,말콤은 급한 상사의 연락을 받고 일을 해결하기 위해 자신의 개인 공구를 챙겨 급히 ...,말콤은 쉬는 날 아픈 아내의 병원에 간병을 와있었네. 상사의 연락을 받고 계단을 내...,말콤은 급한 상사의 연락을 받고 일을 해결하기 위해 공구를 챙겨 급히 문을 나섰네....,말콤은 쉬는 날 아픈 아내의 병원에 간병을 와있었네. 상사의 연락을 받고 계단을 내...,정전 계단 아내 전기 말콤 죽음 간병 병원
1,2,11,"A와 B와 C는 오랜 친구 사이네. A의 생일날 A는 B,C를 집에 초대했네. A의...","A,B,C는 과거에 조난을 당한 적이 있었네. 먹을 것이 있어서 셋은 팔을 하나씩 ...","A와 B와 C는[셋은] 오랜 친구 사이네. A의 생일날 A는 B,C[둘]를 집에 초...","AB,C는 과거에 여행을 가서 조난을 당한 적이 있었네. 먹을 것이 없어서 셋은 팔...",A B C 친구 여행 생일 과거 박수 장애 분노 눈 조난 소리
2,3,28,바다거북 마을 사람들은 거북꽃을 무척이나 좋아한다네. 무척 아름다운 꽃으로 마을 사...,전쟁에 휘말린 바다거북 마을 주변에는 무수히 많은 지뢰가 묻혀 있었고 수많은 피해가...,바다거북 마을 사람들은 거북꽃을 무척이나 좋아[아낀다]한다네. 거북꽃은 무척 아름다...,전쟁 때문에 마을 주변에는 많은 지뢰가 묻혀있었다. 마을 사람들은 마을을 탈출[이주...,지뢰 2년 안전 전쟁 마을 이주 떠나 피난 거북꽃
3,4,22,스산한 분위기에서 한 사람이 소설을 썼네. 소설이 출간되자 이 소설가는 살인죄로 구...,독자는 강압수사를 한 형사 작가는 억울하게 죄를 뒤집어쓰고 거짓 진술서를 쓴 사람이...,스산한 분위기에서 한 사람이 소설을 썼네. 소설이 출간되자 이 소설가는 살인죄로 구...,독자는 강압수사를 한 형사 작가는 억울하게 죄를 뒤집어쓰고 거짓[가짜] 진술서[소설...,협박 허위 살인 사람 선고 독자 소설 거짓 형사 강요 진술
4,5,20,한 여자가 호텔앞에서 고뇌하고 있네. 단지 몇 걸음만 더 가게되면 모든 것을 잃어버...,그 여자는 친구들과 브루마블 보드게임을 즐기고 있던 상황이네. 게임을 곧 종료할 상...,한 여자가[사람이] 호텔앞에서 고뇌하고 있네. 단지 몇 걸음만 더 가게되면 모든 것...,그 여자는[사람은] 친구들과 브루마블 보드게임을 즐기고 있던 상황이네. 게임을 곧 ...,여자 친구 보드 건설 게임 호텔 브루마블 보드게임
5,6,29,한 여학생이 멀쩡한 팔에 깁스를 하고 시험을 보러 나섰네. 신체 건강하고 다친 적도...,이 여학생은 외국어 구두시험을 앞두고 멀쩡한 팔에 일부로 깁스를 했네. 이렇게 하면...,한 여학생이[학생이] 멀쩡한 팔에 깁스를 하고 시험을 보러 나섰네[갔네]. 신체 건...,이 여학생은[학생은] 외국어 구두시험을 앞두고 멀쩡한 팔에 일부로 깁스를 했네. 이...,건강 깁스 관심 예상 시험여자학생


# 사용자 정의 함수

In [ ]:
def get_gpt_answer(input_q):
    return_answer = ''
    while 1:
        input_ids = torch.LongTensor(koGPT2_TOKENIZER.encode(Q_TKN + input_q + SENT + Q_num + A_TKN + return_answer)).unsqueeze(dim=0).to(device)
        pred = model(input_ids)
        pred = pred.logits
        gen = koGPT2_TOKENIZER.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().cpu().numpy().tolist())[-1]
        if gen == EOS:
            break
        return_answer += gen.replace("▁", " ")

    return return_answer


def get_similarity_scores_cleaning_version(dataFrame, Q_num, input):
    i = df[df['원래번호'] == Q_num].index[0]

    new_question = dataFrame['유사도문제']
    new_answer = dataFrame['유사도답']
    main_keyword = dataFrame['유사도키워드']

    sentences = [input, new_question[i], new_answer[i], main_keyword[i]]
    # print(sentences)
    encoded_input = roberta_tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
    model_output = roberta_model(**encoded_input)
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
    score_question = round(float(F.cosine_similarity(sentence_embeddings[0].unsqueeze(0), sentence_embeddings[1].unsqueeze(0))), 4)
    score_answer = round(float(F.cosine_similarity(sentence_embeddings[0].unsqueeze(0), sentence_embeddings[2].unsqueeze(0))), 4)
    score_keyword = round(float(F.cosine_similarity(sentence_embeddings[0].unsqueeze(0), sentence_embeddings[3].unsqueeze(0))), 4)

    return score_question, score_answer, score_keyword

# 모델 불러오기(유사도 검증용 roberta + kogpt2)

In [ ]:
# 모델 불러오기 + 사용자 정의함수 추가
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# Load model from HuggingFace Hub
roberta_tokenizer = AutoTokenizer.from_pretrained('ddobokki/klue-roberta-small-nli-sts')
roberta_model = AutoModel.from_pretrained('ddobokki/klue-roberta-small-nli-sts')

roberta_model.eval()

# 모델 불러오기
Q_TKN, A_TKN, BOS, EOS, MASK, SENT, PAD = "<usr>", "<sys>", '</s>', '</s>', '<unused0>', '<unused1>', '<pad>'
koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
            bos_token=BOS, eos_token=EOS, unk_token='<unk>',
            pad_token=PAD, mask_token=MASK)
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

# GPU, CPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 사용자 정의 토큰 추가
custom_tokens = []
for i in [1,11,28,22,29,20]:
    txt='<문제' + str(int(i)) + '>'
    custom_tokens.append(txt)

koGPT2_TOKENIZER.add_special_tokens({'additional_special_tokens': custom_tokens})
model.resize_token_embeddings(len(koGPT2_TOKENIZER))

model.to(device)
# 모델 불러오기
checkpoint = torch.load(model_path, map_location=torch.device(device))
model.load_state_dict(checkpoint['model_state_dict'])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/530 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

<All keys matched successfully>

# 모델 테스트 실행

In [ ]:
with torch.no_grad():
    while 1:
        inputs = input("문제번호-질문 > ").strip().split('-')
        if inputs[0] == "quit":
            break
        q = inputs[1].upper().strip()
        num = inputs[0].strip()
        a = ""
        Q_num = '<문제' + num + '>'
        # 유사도 검증
        sim_score_question, sim_score_answer, sim_score_keyword  = get_similarity_scores_cleaning_version(df, int(num), q)
        if sim_score_answer > 0.70:
            a = '이젠 정답을 공개해도 되겠군. 아주 잘했네 브리튼'
        elif sim_score_answer > 0.80:
            a = '완벽하네. 브리튼. 자네는 정말 대단하군.'
        elif (sim_score_question < 0.21) & (sim_score_answer < 0.21) & (sim_score_keyword < 0.165):
            a = '정답과 관련이 없는 질문이네. 다른 질문을 해보게나 브리튼'
        elif (sim_score_question < 0.3) & (sim_score_answer < 0.3) & (sim_score_keyword < 0.165):
            a = '그건 중요하지 않네 브리튼 다른 질문을 해보게'
        else :
            i = df[df['원래번호'] == int(num)].index[0]
            input_q = '정답 = ' + df.loc[i,'정답'] + '질문 = ' + q
            a = get_gpt_answer(input_q)
        print("Chatbot > {}".format(a.replace("   ", " ").replace("  ", " ").strip()))
        print('         판단근거 = 문제유사도 :', sim_score_question,'정답유사도 :', sim_score_answer,'키워드 유사도 :', sim_score_keyword )

문제번호-질문 > 1-왜 넘어진건데?
Chatbot > 정답과 관련이 없는 질문이네. 다른 질문을 해보게나 브리튼
         판단근거 = 문제유사도 : 0.1959 정답유사도 : -0.0118 키워드 유사도 : -0.0991
문제번호-질문 > 11-A는 어떤 소리를 들었어?
Chatbot > A는 박수소리를 들었네
         판단근거 = 문제유사도 : 0.2015 정답유사도 : 0.3256 키워드 유사도 : 0.2394
문제번호-질문 > 11-박수 소리가 중요해?
Chatbot > 맞네 거의 다 왔군
         판단근거 = 문제유사도 : 0.087 정답유사도 : 0.1263 키워드 유사도 : 0.2834
문제번호-질문 > 1-말콤의 아내는 어디에 있는데?
Chatbot > 아내는 병원에 있다네
         판단근거 = 문제유사도 : 0.4515 정답유사도 : 0.5893 키워드 유사도 : 0.504
문제번호-질문 > 1-아내가 죽었다는 안내방송을 들은거야?
Chatbot > 아니네 말콤은 아내를 해치지 않았네
         판단근거 = 문제유사도 : 0.4066 정답유사도 : 0.505 키워드 유사도 : 0.3959
문제번호-질문 > 1- 말콤이 왜 넘어진건데?
Chatbot > 말콤은 어두워서 넘어졌다네
         판단근거 = 문제유사도 : 0.4757 정답유사도 : 0.4241 키워드 유사도 : 0.3148
문제번호-질문 > quit
